In [209]:
import pandas as pd
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin

# Upload the data file

In [176]:
from google.colab import files
uploaded = files.upload()

Saving Text_Classifier.csv to Text_Classifier (7).csv


# Load the data into a pandas dataframe

In [210]:
data = pd.read_csv("Text_Classifier (7).csv")
print(data.head())
     

   Doc           Words Author
0    1  W1 W2 W3 W4 W5      C
1    2     W1 W1 W4 W3      C
2    3        W1 W2 W5      C
3    4  W5 W6 W1 W2 W3      W
4    5        W4 W5 W6      W


# Print the shape and info of the data

In [211]:
print(data.shape)
print(data.info())
print(data['Author'].value_counts())

(7, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Doc     7 non-null      int64 
 1   Words   7 non-null      object
 2   Author  7 non-null      object
dtypes: int64(1), object(2)
memory usage: 296.0+ bytes
None
C     3
W     2
F     2
Name: Author, dtype: int64


# Create our list of stopwords

In [213]:
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors

In [212]:
parser = English()

# Define a function to tokenize text using the spacy parser and remove stopwords and punctuation

In [216]:
def spacy_tokenizer(text):
    # Parse the text using the spacy parser
    tokens = parser(text)

    # Remove stopwords and punctuation from the tokens
    tokens = [token.lemma_.lower().strip() if token.lemma_ != "-PRON-" else token.lower_ for token in tokens if token.text not in stop_words and token.text not in string.punctuation]

    # Return the cleaned tokens
    return tokens

# Define a transformer class for cleaning text

In [225]:
class TextPreprocessor(TransformerMixin):
    def transform(self, X, **transform_params):
        # Apply the clean_text function to each element of X
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Define a function to clean the text


In [226]:
def clean_text(text):
    return text.strip().lower()


# Create a Bag of Words vectorizer using the spacy tokenizer

In [227]:
bow_vectorizer = CountVectorizer(tokenizer=spacy_tokenizer, ngram_range=(1,1))

# Create a TF-IDF vectorizer using the spacy tokenizer

In [228]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=spacy_tokenizer)

# Split the data into training and testing sets


In [246]:
X_train, X_test, y_train, y_test = train_test_split(data['Words'], data['Author'], test_size=0.5)


# Logistic Regression Classifier

In [247]:
classifier = LogisticRegression()

# Define a pipeline that preprocesses text, vectorizes it using the bag-of-words approach, and classifies it using logistic regression


In [248]:
pipe = Pipeline([
    ('preprocessor', TextPreprocessor()),
    ('vectorizer', bow_vectorizer),
    ('classifier', classifier)
])

# Fit the pipeline to the training data

In [250]:
pipe.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('preprocessor',
                 <__main__.TextPreprocessor object at 0x7f08d39ffca0>),
                ('vectorizer',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7f08cfb16d30>)),
                ('classifier', LogisticRegression())])

# Predict the authors of the test data using the trained model

In [251]:
predicted_authors = pipe.predict(X_test)

# Predict the author of new data using the trained model

In [252]:
new_text = ["w1 w4 w6 w5 w3"]

In [253]:
predicted_author = pipe.predict(new_text)
print("Predicted author:", predicted_author)

Predicted author: ['W']
